## Stiching Python object

- We use the previously stiching coordinates to stitch the segmented cells from segmentation masks of each position
- We use the previously stiching coordinates to stitch the detected point cloud of each round to ensure not counting twice the same spots

- For visualisation purposes it also possible to stitch fish signal images


## Stitching of segmentation mask

Cell of each position was segmented independly for memory optmization. When stitching the segmentation mask, the overlapping cells from different position are merged.\
When merging segmented cells, their instance cell segmentation ID changes from the one from the local position mask to the global stitched mask.

The fonction **stich_segmask** will stitch segmentation mask using coordinate previously computed

**stich_segmask** saves the new stitched mask in a separated folder and returns a dataframe indicating the correspondance between instance cell segmentation ID between till masks and stitched masks

In [1]:
import numpy as np
from pathlib import Path
from autofish_analysis.stitching import stich_segmask

In [2]:
## Path to the folder containing all the rounds
folder_of_rounds = "/media/tom/Transcend/autofish_test_stiching/"

## path to the mask of the position tiles
path_mask = "/media/tom/Transcend/autofish_test_stiching/segmentation_mask"

## indicate where to save the stiched mask
path_to_save_mask = "/media/tom/Transcend/autofish_test_stiching/segmentation_mask_stitch"

## overlap threshold above two cells are considered the same when stitching
iou_threshold = 0.10 

### load previously computed stitching coordinates
dict_stitch_img = np.load(Path(folder_of_rounds) / "dict_stitch_img.npy",
                          allow_pickle=True).item()

## image dimention parameters
image_shape=[38, 2048, 2048]
nb_tiles_x=3
nb_tiles_y=1

In [3]:
dict_stitch_img = np.load(Path(folder_of_rounds) / "dict_stitch_img.npy",
                          allow_pickle=True).item()
df_matching_new_cell_label = stich_segmask(dict_stitch_img, 
                  path_mask = path_mask,
                  path_to_save_mask = path_to_save_mask,
                  image_shape=image_shape,
                  nb_tiles_x = nb_tiles_x,
                  nb_tiles_y=nb_tiles_y,
                  iou_threshold = 0.10)

In [4]:
df_matching_new_cell_label['img1']

**the cell_id_local_position** is the cell ID in the orignal segmented tile at postion X\
**the cell_id_stitched_mask** is the cell ID in the stitched mask

In [6]:
np.save(Path(folder_of_rounds) / "df_matching_new_cell_label.npy", df_matching_new_cell_label)

## Stitching of point clouds

The function **dico_spots_registered_stitch_df** stitches the point cloud from the different tiles


In [11]:
import autofish_analysis.stitching
importlib.reload(autofish_analysis.stitching)
from autofish_analysis.stitching import  stich_dico_spots

In [12]:


## define the correspondance between round and RNA species
dict_round_gene={
        'r1': "gene1",
        'r3': "gene2",
        }


### load previously registered spots detection
dict_spots_registered_df = np.load(Path(folder_of_rounds) / "dict_spots_registered_df_r1_with_cell.npy",
                                allow_pickle=True).item()



dico_spots_registered_stitch_df = stich_dico_spots(
    dict_spots_registered_df = dict_spots_registered_df,
                     dict_stitch_img = dict_stitch_img,
                     dict_round_gene=dict_round_gene,
                     image_shape=image_shape,
                 nb_tiles_x=nb_tiles_x,
                 nb_tiles_y=nb_tiles_y,
                 df_matching_new_cell_label = df_matching_new_cell_label)

np.save(Path(folder_of_rounds) /  "dico_spots_registered_stitch_df.npy",
        dico_spots_registered_stitch_df)


In [13]:
dico_spots_registered_stitch_df['img0']

### Stich fluorescent signal

Stitching the fish images is only for vizualisation purposes as all analysis are computed on local postion tile.
It enable to reduce computational cost. 

Still it is possible to stitch fish signal of the reference round using the previously computed stitching coordiante

In [17]:
import autofish_analysis.stitching
importlib.reload(autofish_analysis.stitching)
from autofish_analysis.stitching import  stich_from_dico_img

In [19]:
image_shape=[38, 2048, 2048]
nb_tiles_x=3
nb_tiles_y=1
for image_name in dict_stitch_img:
    dict_stitch = dict_stitch_img[image_name]
    final_masks = stich_from_dico_img(dict_stitch,
                      path_mask = "/media/tom/Transcend/autofish_test_stiching/r1",
                        regex = "*_ch0*tif*", ## regular expression for the images to stitch
                        image_shape=image_shape,
                            nb_tiles_x=nb_tiles_x,
                            nb_tiles_y=nb_tiles_y,)
    
    np.save(Path(folder_of_rounds) / Path("/media/tom/Transcend/autofish_test_stiching/stitching_r1") / image_name,
            final_masks)


In [22]:
from matplotlib import pyplot as plt
from matplotlib import pyplot as plt
from skimage.exposure import rescale_intensity


input_img = np.amax(final_masks, 0)
pa_ch1, pb_ch1 = np.percentile(input_img, (1, 99))
input_img = rescale_intensity(input_img, in_range=(pa_ch1, pb_ch1), out_range=np.uint8)

plt.imshow(input_img)
